In [1]:
import json
import pandas as pd
from sklearn.model_selection import train_test_split

In [2]:
import random
import numpy as np
import torch 
def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)

set_seed(42)

In [3]:
path_to_json_file = '/scratch/ns5287/sarcasm_detection_shared_task_twitter_training.jsonl'
import json 

with open(path_to_json_file, 'r') as j:
     json_data = [json.loads(line) for line in j]

In [4]:
from tqdm import tqdm
accurate = 0
tot_tweets = 0

tweets=[]
labels=[]
for tweet in tqdm(json_data):
    tot_cont = ""
    for con in tweet['context']:
        tot_cont+=con 
    tot_cont+=tweet['response']
    tweets.append(tot_cont)
    labels.append(tweet['label'])

100%|██████████| 5000/5000 [00:00<00:00, 391947.07it/s]


In [5]:
import pandas as pd

data = {'target_text':labels,
        'source_text':tweets}
 
train = pd.DataFrame(data)
 
train.head()

,target_text,source_text
0,SARCASM,A minor child deserves privacy and should be k...
1,SARCASM,@USER @USER Why is he a loser ? He's just a Pr...
2,SARCASM,Donald J . Trump is guilty as charged . The ev...
3,SARCASM,Jamie Raskin tanked Doug Collins . Collins loo...
4,SARCASM,Man ... y ’ all gone “ both sides ” the apocal...


In [6]:
path_to_json_file = '/scratch/ns5287/sarcasm_detection_shared_task_twitter_testing.jsonl'
import json 

with open(path_to_json_file, 'r') as j:
     json_data = [json.loads(line) for line in j]

In [7]:
from tqdm import tqdm

tweets=[]
labels=[]
for tweet in tqdm(json_data):
    tot_cont = ""
    for con in tweet['context']:
        tot_cont+=con 
    tot_cont+=tweet['response']
    tweets.append(tot_cont)
    labels.append(tweet['label'])

100%|██████████| 1800/1800 [00:00<00:00, 474081.46it/s]


In [8]:
import pandas as pd

data = {'target_text':labels,
        'source_text':tweets}
 
test = pd.DataFrame(data)
 
print(len(test))

1800


In [9]:
import random

train = train.sample(frac = 1)
train_data = train[:4500]
val_data = train[4500:]

In [10]:
from simplet5 import SimpleT5

Global seed set to 42


In [11]:
model = SimpleT5()
model.from_pretrained("byt5","google/byt5-base")

In [ ]:
model.train(train_df=train_data, 
            eval_df=val_data, 
            source_max_token_len = 1024, 
            target_max_token_len = 128,
            batch_size = 2,
            max_epochs = 10,
            use_gpu = True,
            outputdir = "BASE_epoch_10_outputs_Twitter",
            early_stopping_patience_epochs = 0
            )

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
Set SLURM handle signals.

  | Name  | Type                       | Params
-----------------------------------------------------
0 | model | T5ForConditionalGeneration | 581 M 
-----------------------------------------------------
581 M     Trainable params
0         Non-trainable params
581 M     Total params
2,326.613 Total estimated model params size (MB)


Validation sanity check: 0it [00:00, ?it/s]

/home/ns5287/.local/lib/python3.8/site-packages/torch/utils/data/dataloader.py:487: UserWarning: This DataLoader will create 2 worker processes in total. Our suggested max number of worker in current system is 1, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(
/home/ns5287/.local/lib/python3.8/site-packages/pytorch_lightning/trainer/data_loading.py:132: UserWarning: The dataloader, val_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 48 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(
Global seed set to 42
/home/ns5287/.local/lib/python3.8/site-packages/pytorch_lightning/trainer/data_loading.py:132: UserWarning: The datalo

Training: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

In [ ]:
!nvidia-smi

In [10]:
from simplet5 import SimpleT5
model = SimpleT5()

Global seed set to 42


In [15]:
pwd

'/scratch/ns5287'

In [17]:
model.load_model('byt5', '/scratch/ns5287/BASE_epoch_10_outputs_Twitter/simplet5-epoch-4-train-loss-0.0464-val-loss-0.0452')

In [ ]:
from tqdm import tqdm
count=0
value=0
lis=[]
for index, row in tqdm(test.iterrows()):
    ans = model.predict(row['source_text'])
    if ans[0]==row['target_text']:
        count+=1
    if value%100==0:
        print(value,count)
    value+=1
    lis.append(ans[0])

print(count*100/1800)

1it [00:06,  6.03s/it]

0 0


101it [10:52,  6.03s/it]

100 67


201it [22:04,  8.18s/it]

...

1001it [1:49:02,  6.61s/it]

1000 665


1101it [1:58:32,  5.30s/it]

1100 732


1176it [2:06:39,  6.08s/it]

In [ ]:
print("Accuracy on 10 epochs:", count*100/1800)